In [1]:
import time
import pickle
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [6]:
from dsgtools import azure
extract_12 = pd.read_adls('Analytics/Personal Folders/liuwei01/Prj2023/origination_extract_Joe/origination_extract_12M_202101_202112.csv', 
                      reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(extract_12.shape)

(3618335, 52)


In [7]:
extract_12 = extract_12[extract_12.AnalysisCreditBad.notnull()]
print(extract_12.shape)
extract_12["AnalysisCreditBad"] = extract_12["AnalysisCreditBad"].astype(int)

(3272574, 52)


In [8]:
bivariate(extract_12.account_type_reported, extract_12.AnalysisCreditBad)

tag                   AnalysisCreditBad                               \
stats                                 N      PctN      Sum      Mean   
account_type_reported                                                  
001                            745671.0  0.227855   5602.0  0.007513   
002                             49732.0  0.015197   3232.0  0.064988   
003                           2086890.0  0.637691  67807.0  0.032492   
004                            209066.0  0.063884   8997.0  0.043034   
005                               565.0  0.000173      1.0  0.001770   
006                            168715.0  0.051554   8110.0  0.048069   
099                             11935.0  0.003647   5099.0  0.427231   
Missing                             0.0  0.000000      0.0  0.000000   
Total                         3272574.0  1.000000  98848.0  0.030205   

tag                                        
stats                       WoE        IV  
account_type_reported                      
001                   -1.414546  0.249687  
002                    0.802722  0.014485  
003                    0.075340  0.003751  
004                    0.367302  0.010277  
005                   -2.865981  0.000480  
006                    0.483223  0.015193  
099                    3.175915  0.156986  
Missing                0.000000  0.000000  
Total                  0.000000  0.450858

In [21]:
freq(extract_12.InsufficientBusiness, extract_12.InsufficientRep, cross = False, observed=True)

Count       Pct  Cuml Count  \
InsufficientBusiness InsufficientRep                                    
0                    1                1896305.0  0.579454   1896305.0   
                     0                1375730.0  0.420382   3272035.0   
1                    1                    393.0  0.000120   3272428.0   
                     0                    146.0  0.000045   3272574.0   

                                      Cuml Pct  
InsufficientBusiness InsufficientRep            
0                    1                0.579454  
                     0                0.999835  
1                    1                0.999955  
                     0                1.000000

In [22]:
extract_12 = extract_12[(extract_12.InsufficientBusiness == "0") & (extract_12.InsufficientRep=="0")]
print(extract_12.shape)
extract_12 = extract_12[['account_type_reported',
  'UniqueIdentifier',
 'ArchiveDate',
 'BusinessState',
 'BusinessZipcode',
 'AnalysisCreditBad',
]]

(1375730, 52)


## Data cleaning

In [3]:
freq(extract_12.AnalysisCreditBad)

,Count,Pct,Cuml Count,Cuml Pct
AnalysisCreditBad,,,,
0,3173726.0,0.877123,3173726.0,0.877123
1,98848.0,0.027319,3272574.0,0.904442
Missing,345761.0,0.095558,3618335.0,1.000000


In [4]:
extract_12 = extract_12[extract_12.AnalysisCreditBad.notnull()]
print(extract_12.shape)
extract_12["AnalysisCreditBad"] = extract_12["AnalysisCreditBad"].astype(int)

(3272574, 52)


In [5]:
bivariate(extract_12.ArchiveDate, extract_12.AnalysisCreditBad)

tag         AnalysisCreditBad                                                 
stats                       N      PctN      Sum      Mean       WoE        IV
ArchiveDate                                                                   
202101               301755.0  0.092207   4556.0  0.015098 -0.708884  0.033709
202102               397448.0  0.121448   5530.0  0.013914 -0.791791  0.053481
202103               408901.0  0.124948   6845.0  0.016740 -0.604000  0.034691
202104               310799.0  0.094971   7326.0  0.023572 -0.254790  0.005480
202105               240389.0  0.073456   7533.0  0.031337  0.037946  0.000108
202106               233503.0  0.071351   8697.0  0.037246  0.216814  0.003718
202107               228185.0  0.069726   9488.0  0.041580  0.331414  0.008974
202108               219631.0  0.067113   9597.0  0.043696  0.383254  0.011846
202109               241325.0  0.073742  10520.0  0.043593  0.380778  0.012833
202110               232326.0  0.070992   9957.0  0.042858  0.363011  0.011131
202111               238467.0  0.072868   9774.0  0.040987  0.316419  0.008486
202112               219845.0  0.067178   9025.0  0.041052  0.318067  0.007912
Missing                   0.0  0.000000      0.0  0.000000  0.000000  0.000000
Total               3272574.0  1.000000  98848.0  0.030205  0.000000  0.192369

In [7]:
freq(extract_12.InsufficientBusiness, extract_12.InsufficientRep, cross = False, observed=True)

Count       Pct  Cuml Count  \
InsufficientBusiness InsufficientRep                                    
0                    1                1896305.0  0.579454   1896305.0   
                     0                1375730.0  0.420382   3272035.0   
1                    1                    393.0  0.000120   3272428.0   
                     0                    146.0  0.000045   3272574.0   

                                      Cuml Pct  
InsufficientBusiness InsufficientRep            
0                    1                0.579454  
                     0                0.999835  
1                    1                0.999955  
                     0                1.000000

In [8]:
extract_12.to_adls('Analytics/RnD Projects/Product RnD/Business/LexisNexis (2023 Model revalidation)/input/origination_extract_12mth_w_perf_202101_202112.parquet', 
                     format = '.parquet', overwrite = True)

In [2]:
extract_18 = pd.read_adls('Analytics/Personal Folders/dobsti01/2023 Model Revalidation/Data/Full_Tradeline_Rollup_18M/part-00000-6bab30aa-b3ad-43b5-a639-5f89892cdee3-c000.csv', 
                      reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(extract_18.shape)
freq(extract_18.AnalysisCreditBad)

(3618335, 53)


,Count,Pct,Cuml Count,Cuml Pct
AnalysisCreditBad,,,,
0,2101956.0,0.580918,2101956.0,0.580918
1,138929.0,0.038396,2240885.0,0.619314
Missing,1377450.0,0.380686,3618335.0,1.000000


In [3]:
extract_18 = extract_18[extract_18.AnalysisCreditBad.notnull()]
freq(extract_18.InsufficientBusiness, extract_18.InsufficientRep, cross = False, observed=True)

Count       Pct  Cuml Count  \
InsufficientBusiness InsufficientRep                                    
0                    1                1385675.0  0.618361   1385675.0   
                     0                 854820.0  0.381465   2240495.0   
1                    1                    307.0  0.000137   2240802.0   
                     0                     83.0  0.000037   2240885.0   

                                      Cuml Pct  
InsufficientBusiness InsufficientRep            
0                    1                0.618361  
                     0                0.999826  
1                    1                0.999963  
                     0                1.000000

In [5]:
extract_18.to_adls('Analytics/RnD Projects/Product RnD/Business/LexisNexis (2023 Model revalidation)/input/origination_extract_18mth_w_perf_202101_202112.parquet', 
                     format = '.parquet', overwrite = True)

## get the dataframe ready

In [3]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 11657 (2023 Model revalidation)/Processing/"

In [11]:
innovis = pd.read_adls(path + "innovis_score_2021_new_joe.csv", reader = pd.read_csv)
innovis = innovis[["acctno", "generic_score_v2", "innovis_scoreable"]]
print(innovis.shape)

(1375876, 3)


In [12]:
cfbp = pd.read_adls(path + "ln_11657_cons1_input_0320_slim_cfpb_race", reader = pd.read_csv)
cfbp["CFPB_Race_Estimate_80"] = np.select([ 
                                         cfbp["pr_white"].gt(0.80), 
                                         cfbp["pr_black"].gt(0.80), 
                                         cfbp["pr_hispanic"].gt(0.80), 
                                         cfbp["pr_api"].gt(0.80), 
                                         cfbp["pr_aian"].gt(0.80), 
                                         cfbp["pr_mult_other"].gt(0.80), ], 
                                        ["White", "Black", "Hispanic", "API", "AIAN", "Mult_Other"], 
                                        default="No Prob. > 0.80 or Prob. Not Available", )
cfbp = cfbp[['inputs.account', 'CFPB_Race_Estimate_80']]
print(cfbp.shape)

(1375876, 2)


In [10]:
use = ['accountnumber', 'historydateyyyymm', 'bus_company_name', 'powid', 'proxid', 'seleid', 'orgid', 'ultid', 
       'businessrecordtimeoldest','sbfehitindex', 'sbfetimeoldest', 'model1score', "model2score"]
part1 = "ln_11657_bus_rep1_input_0320_pt1_sba_v21_sbfe_busshell_31_w20230320-165230-3.csv.gz"
part2 = "ln_11657_bus_rep1_input_0320_pt1_sba_v21_sbfe_busshell_31_w20230320-223541.csv.gz"
part3 = "ln_11657_bus_rep1_input_0320_pt2_sba_v21_sbfe_busshell_31_rew20230320-223558.csv.gz"
part4 = "ln_11657_bus_rep1_input_0320_pt2_sba_v21_sbfe_busshell_31_w20230320-165245.csv.gz"
part5 = "ln_11657_bus_rep1_input_0320_pt3_sba_v21_sbfe_busshell_31_w20230320-165501.csv.gz"
part6 = "ln_11657_bus_rep1_input_0320_pt4_sba_v21_sbfe_busshell_31_w20230320-165514.csv.gz"
part7 = "ln_11657_bus_rep1_input_0320_pt5_sba_v21_sbfe_busshell_31_w20230320-165523.csv.gz"


part1_df = pd.read_adls(path + part1, reader = pd.read_csv, compression = "gzip", encoding = 'iso-8859-1', usecols = use)
part2_df = pd.read_adls(path + part2, reader = pd.read_csv, compression = "gzip", encoding = 'iso-8859-1', usecols = use)
part3_df = pd.read_adls(path + part3, reader = pd.read_csv, compression = "gzip", encoding = 'iso-8859-1', usecols = use)
part4_df = pd.read_adls(path + part4, reader = pd.read_csv, compression = "gzip", encoding = 'iso-8859-1', usecols = use)
part5_df = pd.read_adls(path + part5, reader = pd.read_csv, compression = "gzip", encoding = 'iso-8859-1', usecols = use)
part6_df = pd.read_adls(path + part6, reader = pd.read_csv, compression = "gzip", encoding = 'iso-8859-1', usecols = use)
part7_df = pd.read_adls(path + part7, reader = pd.read_csv, compression = "gzip", encoding = 'iso-8859-1', usecols = use)

stacked = pd.concat([part1_df, part2_df, part3_df, part4_df, part5_df, part6_df, part7_df], ignore_index = True)
print(stacked.shape)
stacked.to_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 11657 (2023 Model revalidation)/Processing/temp_parquet/sba_12m_slim.parquet", 
                format = '.parquet', overwrite = True)

(3272574, 13)


In [23]:
extract_12 = extract_12.merge(cfbp, left_on = "UniqueIdentifier", right_on = "inputs.account")
extract_12 = extract_12.merge(innovis, left_on = "UniqueIdentifier", right_on = "acctno")
extract_12 = extract_12.merge(stacked, left_on = "UniqueIdentifier", right_on = "accountnumber")
print(extract_12.shape)

(1375730, 24)


In [24]:
extract_12['mths_firstseen'] = extract_12[['businessrecordtimeoldest', 'sbfetimeoldest']].max(axis = 1)
extract_12['new_business'] = np.where(extract_12['mths_firstseen']<0, -998, 
                                    np.where(extract_12['mths_firstseen'] <= 12, 1, 0))
freq(extract_12['new_business'])

,Count,Pct,Cuml Count,Cuml Pct
new_business,,,,
-998,471629.0,0.342821,471629.0,0.342821
0,693296.0,0.503948,1164925.0,0.846769
1,210805.0,0.153231,1375730.0,1.000000


In [25]:
## Social deprivation index
sdi = pd.read_adls(path + "SDI_ACS2015_zctaallvars (1).xlsx", reader = pd.read_excel, sheet_name = "ACS2015_zctaallvars", dtype = str)
print(sdi.shape)
# ZCTA is the zip code and SDI_Score is the SDI.  
# There may be a very small amount of zip’s that don’t have an SDI due to not having the data available for that, but it should be a very small amount.
# Match to zip5: you should do that and front load with a 0 for the 4 digit values
sdi = sdi[["zcta", "sdi_score"]]
sdi['zcta_padded'] = sdi['zcta'].apply(lambda x: x.zfill(5))
sdi['sdi_score'] = sdi['sdi_score'].astype(float)
extract_12 = extract_12.merge(sdi, how = "left", left_on = "BusinessZipcode", right_on = "zcta_padded")
print(extract_12.shape)
extract_12["sdi_score"].describe()

(32989, 29)
(1375730, 29)


count    1.363966e+06
mean     4.464023e+01
std      2.847920e+01
min      1.000000e+00
25%      2.000000e+01
50%      4.200000e+01
75%      6.800000e+01
max      1.000000e+02
Name: sdi_score, dtype: float64

In [26]:
# if [Sdi Score] > 80 then 'Depressed'ELSEIF [Sdi Score] > 20 THEN 'Average'
# ELSEIF [Sdi Score] >=0 THEN 'Affluent'
# ELSE 'Other'
# The higher the score the more depressed socioeconomic level.  
# Also, each value represents roughly 1% of the population of the US, which worked pretty good using our Prospect Generator database
extract_12["sdi_flag"] = np.where(extract_12["sdi_score"] >80, "Depressed", 
                                  np.where(extract_12["sdi_score"] > 20, "Average", 
                                          np.where(extract_12["sdi_score"] >=0, "Affluent", "_Other")))
freq(extract_12["sdi_flag"])

,Count,Pct,Cuml Count,Cuml Pct
sdi_flag,,,,
Affluent,356310.0,0.258997,356310.0,0.258997
Average,806292.0,0.586083,1162602.0,0.845080
Depressed,201364.0,0.146369,1363966.0,0.991449
_Other,11764.0,0.008551,1375730.0,1.000000


In [27]:
northeast = ['NH', 'VT' , 'MA' , 'RI' , 'CT' , 'NY' , 'NJ' , 'PA' , 'ME' ]
midwest = ['MI' , 'IN' , 'WI' , 'IL' , 'MN' , 'IA' , 'MO' , 'ND' , 'SD' , 'NE' , 'KS', 'OH' ]
south = ['MD' , 'VA' , 'WV' , 'KY' , 'NC' , 'SC' , 'TN' , 'GA' , 'FL' , 'AL' , 'MS' , 'AR' , 'LA' , 'TX' , 'OK' , 'DE' ]
west = ['ID' , 'WY' , 'CO' , 'NM' , 'AZ' , 'UT' , 'NV' , 'CA' , 'OR' , 'WA' , 'AK' , 'HI' , 'MT' ]

extract_12["region_flag"] = np.where(extract_12["BusinessState"].isin(northeast), "Northeast", 
                                     np.where(extract_12["BusinessState"].isin(midwest), "Midwest", 
                                             np.where(extract_12["BusinessState"].isin(south), "South", 
                                                     np.where(extract_12["BusinessState"].isin(west), "West", "_Other"))))
freq(extract_12["region_flag"])

,Count,Pct,Cuml Count,Cuml Pct
region_flag,,,,
Midwest,203570.0,0.147972,203570.0,0.147972
Northeast,234018.0,0.170105,437588.0,0.318077
South,559207.0,0.406480,996795.0,0.724557
West,375025.0,0.272601,1371820.0,0.997158
_Other,3910.0,0.002842,1375730.0,1.000000


In [28]:
extract_12.columns

Index(['account_type_reported', 'UniqueIdentifier', 'ArchiveDate',
       'BusinessState', 'BusinessZipcode', 'AnalysisCreditBad',
       'inputs.account', 'CFPB_Race_Estimate_80', 'acctno', 'generic_score_v2',
       'innovis_scoreable', 'accountnumber', 'historydateyyyymm',
       'bus_company_name', 'powid', 'proxid', 'seleid', 'orgid', 'ultid',
       'businessrecordtimeoldest', 'sbfehitindex', 'sbfetimeoldest',
       'model1score', 'model2score', 'mths_firstseen', 'new_business', 'zcta',
       'sdi_score', 'zcta_padded', 'sdi_flag', 'region_flag'],
      dtype='object')

In [29]:
keep = ['acctno', 'ArchiveDate', 'AnalysisCreditBad', 'sbfehitindex', 'new_business', 'BusinessState', 'BusinessZipcode', 
        'CFPB_Race_Estimate_80', 'generic_score_v2', 'model2score', 'model1score', 'sdi_score', 'sdi_flag', 'region_flag']
extract_12 = extract_12[keep]
print(extract_12.shape)


(1375730, 14)


In [30]:
extract_12.to_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 11657 (2023 Model revalidation)/Processing/temp_parquet/fair_lending_score_flag_file_approval_rate_new_2021.csv", 
                format = '.csv', overwrite = True, index = False)